In [1]:
import vjp.data as data

In [2]:
second_instance_sampels = data.load_second_instance()

Samples are parsed as Python XML element trees. Queries can be done per-element through [the XPath syntax](https://docs.python.org/3/library/xml.etree.elementtree.html#supported-xpath-syntax). `vjp.data.finall` permits querying an entire list. All results from all queries are added to an output list.

In [22]:
print('Number of instances', len(second_instance_sampels))
print('Number of second instance requests', len(data.findall(second_instance_sampels, ".//partreq[@G='2']/req")))

decisions = data.findall(second_instance_sampels, ".//courtdec[@G='2']/dec")
upheld = data.findall(decisions, ".[@E='1']")
rejected = data.findall(decisions, ".[@E='0']")
print('Number of second instance decisions', len(decisions))
print('Of which upheld', len(upheld))
print('Of which rejected', len(rejected))
print('Other outcomes', len(decisions) - len(upheld) - len(rejected))

Number of instances 219
Number of second instance requests 480
Number of second instance decisions 249
Of which upheld 110
Of which rejected 135
Other outcomes 4
